# Unified Bombcell Runner (Grant)

Single entrypoint replacing batch, single-probe, and NP2.0 rerun notebooks.

In [1]:
CONFIG_FILE = r'C:\Users\user\Documents\github\bombcell\py_bombcell\grant\configs\grant_recording_config_reach15_20260201_session007.json'
RUN_MODE = 'batch'  # batch | single_probe | np20_rerun
TARGET_PROBE = 'B'  # only used for single_probe
OVERWRITE = False


In [ ]:
from pathlib import Path
import subprocess
import sys
import pandas as pd

# Resolve runner path robustly (works from running_BC, grant, or repo root working dirs)
runner_candidates = [
    Path.cwd() / 'run_bombcell_unified.py',
    Path.cwd() / 'running_BC' / 'run_bombcell_unified.py',
    Path.cwd() / 'grant' / 'running_BC' / 'run_bombcell_unified.py',
    Path.cwd() / 'py_bombcell' / 'grant' / 'running_BC' / 'run_bombcell_unified.py',
]
runner = next((p for p in runner_candidates if p.exists()), None)
if runner is None:
    raise FileNotFoundError(
        'Could not find run_bombcell_unified.py from current working directory. '
        f'Checked: {[str(p) for p in runner_candidates]}'
    )
runner = runner.resolve()

cfg_path = Path(CONFIG_FILE)
if not cfg_path.exists():
    raise FileNotFoundError(f'Config file not found: {cfg_path}')

cmd = [sys.executable, str(runner), '--config', str(cfg_path), '--mode', RUN_MODE]
if RUN_MODE == 'single_probe':
    cmd += ['--target-probe', TARGET_PROBE]
if OVERWRITE:
    cmd.append('--overwrite')

print('Runner:', runner)
print('Running:', ' '.join(cmd))

try:
    result = subprocess.run(cmd, check=True, text=True, capture_output=True, cwd=str(runner.parent))
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print('--- stderr ---')
        print(result.stderr)
except subprocess.CalledProcessError as exc:
    print(f'❌ Bombcell runner failed with exit code {exc.returncode}')
    if exc.stdout:
        print('--- stdout ---')
        print(exc.stdout)
    if exc.stderr:
        print('--- stderr ---')
        print(exc.stderr)
    raise

# Verify ROI labels for Phy clusterView when probe_recording_roi is set
sys.path.insert(0, str(runner.parent.parent.resolve()))  # .../py_bombcell/grant
from grant_config import load_grant_config
cfg = load_grant_config(cfg_path)
roi_cfg = cfg.get('probe_recording_roi', {})

if RUN_MODE == 'single_probe':
    probes = [TARGET_PROBE.upper()]
elif RUN_MODE == 'np20_rerun':
    probes = [p.upper() for p in cfg['np20_probes']]
else:
    probes = [p.upper() for p in cfg['probes_all']]

for probe in probes:
    if roi_cfg.get(probe) is None:
        print(f'Probe {probe}: probe_recording_roi not set, skipping ROI label file check.')
        continue

    ks_dir = Path(cfg['probe_kilosort_dirs'][probe])
    roi_tsv = ks_dir / 'cluster_bc_roiLabel.tsv'

    if not roi_tsv.exists():
        raise FileNotFoundError(
            f'Expected ROI label file for probe {probe} was not created: {roi_tsv}. '
            'Set probe_recording_roi in config and rerun.'
        )

    roi_df = pd.read_csv(roi_tsv, sep='\t')
    labels = set(roi_df.get('bc_roiLabel', pd.Series(dtype=str)).dropna().astype(str))
    print(f'Probe {probe}: ROI label file OK -> {roi_tsv} | labels: {sorted(labels)}')

    if not labels.intersection({'IN_ROI', 'OUT_ROI'}):
        raise ValueError(f'ROI label file exists but has no IN_ROI/OUT_ROI labels for probe {probe}: {roi_tsv}')
